# Métodos numéricos para resolver el problema de caida libre

Un paracaidista con una masa de 87.5 kg salta de un globo aerost ́atico fijo.
Tomando en cuenta la expresi ́on de un modelo matem ́atico simple, calcular la
velocidad terminal del paracaidista. Considere que el coeficiente de arrastre es
igual a 13.5 kg/s.

Se presentan los siguientes métodos:
* Euler
* Euler modificado
* Runge-Kutta de orden 4

#### La siguiente celda solo se debe ejecutar la primera vez que se use el notebook para instalar las bibliotecas necesarias.

In [ ]:
# Instalar las bibliotecas
%pip install plotly --q
%pip install nbformat --q
%pip install sympy --q

In [1]:
# Importar las bibliotecas
from sympy import lambdify, sympify # ¡¡¡NO MODIFICAR!!!
from sympy.abc import t, v # ¡¡¡NO MODIFICAR!!!
import plotly.express as px # ¡¡¡NO MODIFICAR!!!
import plotly.graph_objects as go # ¡¡¡NO MODIFICAR!!!
import numpy as np # ¡¡¡NO MODIFICAR!!!

### Definición de constantes y funciones

In [31]:
t_0 = 0 # Tiempo inicial en minutos
t_f = 25 # Tiempo final en minutos
valores_t = np.linspace(t_0, t_f, (t_f-t_0)*15) # Vector de tiempo !!!NO MODIFICAR!!!
gravedad = 9.81 # gravedad en m/s^2

masa = 87.5 # masa en kg
k = 13.5  # coeficiente de arrastre kg/s
C = -masa*gravedad/k # Constante de integración

f_real = f'{gravedad}-{k/masa}*v' # Ecuacion diferencial de la forma dy/dx = f(x, y) ¡¡¡NO MODIFICAR!!!
f_real = lambdify(v, sympify(f_real), 'numpy') # Funcion f(x, y) en formato numpy ¡¡¡NO MODIFICAR!!!
f_sol_analitica = f'{masa*gravedad/k}+{C}*exp(-{k/masa}*t)' # Solucion analitica de la ecuacion diferencial ¡¡¡NO MODIFICAR!!!
f_sol_analitica = lambdify(t, sympify(f_sol_analitica), 'numpy') # Funcion f(x, y) en formato numpy ¡¡¡NO MODIFICAR!!!

### Analitica

In [32]:
def solucion_analitica(f, valores_t, C, k):
    # Evaluar la función
    valores_y = list(map(lambda t: f(t), valores_t)) # Evaluar la función en el vector de tiempo

    # Graficar
    titulo = f'Caida libre<br>Solucion analítica: v(t)={round(masa*gravedad/k,3)}{+round(C,3)}*exp(-{round(k/masa,3)}*t)' # Titulo de la gráfica
    eje_x = 't[s]' # Nombre del eje x
    eje_y = 'v(t)[m/s]' # Nombre del eje y 

    fig = go.Figure() # Crear figura para graficar
    fig.update_layout(title=titulo, title_x=0.5, xaxis_title=eje_x, yaxis_title=eje_y) # Actualizar diseño de la gráfica
    plot_analitica = go.Scatter(x=valores_t, y=valores_y, name='Solución analítica', line=dict(color='blue', width=2), mode='lines', showlegend=True) # Crear gráfica de la solución analítica
    fig.add_trace(plot_analitica) # Agregar gráfica de la solución analítica a la figura
    fig.show() # Mostrar figura
    return plot_analitica # Regresar gráfica de la solución analítica

plot_Analitica = solucion_analitica(f_sol_analitica, valores_t, C, k) # Graficar solución analítica

### Definición de constante para métodos numéricos

In [33]:
h = 0.2 # Tamaño de paso
n_pasos = int(np.ceil((t_f-t_0)/h)) # Número de pasos !!!NO MODIFICAR!!!
y_0 = 0 # Masa inicial !!!NO MODIFICAR!!!

### Euler

In [34]:
def metodo_euler(f, t_0, y_0, h, n_pasos):
    valores_y_Euler = [y_0] # Lista de valores de y
    valores_t = [t_0] # Vector de tiempo
    
    # Iterar sobre el número de pasos y aplicar el método de Euler
    for i in range(n_pasos):
        valores_t.append(valores_t[i] + h) # Agregar el valor de t a la lista
        y_nueva = valores_y_Euler[i] + h * f(valores_y_Euler[i]) # Método de Euler
        valores_y_Euler.append(y_nueva) # Agregar el valor de y a la lista

    # Graficar
    titulo = f'Caida libre<br>Método de Euler con h = {round(h,3)}, n = {n_pasos} y T(0) = {round(y_0,3)}' # ¡¡¡NO MODIFICAR!!!
    eje_x = 't[s]' # Nombre del eje x
    eje_y = 'v(t)[m/s]' # Nombre del eje y

    fig = go.Figure() # ¡¡¡NO MODIFICAR!!!
    fig.update_layout(title=titulo, title_x=0.5, xaxis_title=eje_x, yaxis_title=eje_y) # ¡¡¡NO MODIFICAR!!!
    plot_Euler = go.Scatter(x=valores_t, y=valores_y_Euler, mode='markers', marker=dict(size=8, symbol='diamond'), name='Euler', marker_color='lime', showlegend=True) # ¡¡¡NO MODIFICAR!!!
    fig.add_trace(plot_Euler) # ¡¡¡NO MODIFICAR!!!
    fig.show() # ¡¡¡NO MODIFICAR!!!
    return plot_Euler # ¡¡¡NO MODIFICAR!!!

plot_Euler = metodo_euler(f_real, t_0, y_0, h, n_pasos) # Graficar método de Euler

### Euler modificado

In [35]:
def metodo_euler_mod(f, t_0, y_0, h, n_pasos):
    valores_y_Euler_Mod = [y_0] # Lista de valores de y
    valores_t = [t_0] # Vector de tiempo

    # Iterar sobre el número de pasos y aplicar el método de Euler modificado
    for i in range(n_pasos):
        valores_t.append(valores_t[i] + h)
        y_nueva = valores_y_Euler_Mod[i] + h * f(valores_y_Euler_Mod[i] + h/2 * f(valores_y_Euler_Mod[i])) # ¡¡¡NO MODIFICAR!!!
        valores_y_Euler_Mod.append(y_nueva) # ¡¡¡NO MODIFICAR!!!

    # Graficar
    titulo = f'Caida libre<br>Método de Euler modificado con h = {round(h,3)}, n = {n_pasos} y N(0) = {round(y_0,3)}' # ¡¡¡NO MODIFICAR!!!
    eje_x = 't[s]' # Nombre del eje x
    eje_y = 'v(t)[m/s]' # Nombre del eje y

    fig = go.Figure() # ¡¡¡NO MODIFICAR!!!
    fig.update_layout(title=titulo, title_x=0.5, xaxis_title=eje_x, yaxis_title=eje_y) # ¡¡¡NO MODIFICAR!!!
    plot_Euler_Mod = go.Scatter(x=valores_t, y=valores_y_Euler_Mod, mode='markers', marker=dict(size=8, symbol='triangle-up'), name='Euler modificado', marker_color='darkviolet', showlegend=True) # ¡¡¡NO MODIFICAR!!!
    fig.add_trace(plot_Euler_Mod) # ¡¡¡NO MODIFICAR!!!
    fig.show() # ¡¡¡NO MODIFICAR!!!
    return plot_Euler_Mod # ¡¡¡NO MODIFICAR!!!

plot_Euler_Mod = metodo_euler_mod(f_real, t_0, y_0, h, n_pasos) # Graficar método de Euler modificado

## Runge-Kutta de orden 4

In [36]:
def metodo_RK_4(f, t_0, y_0, h, n_pasos):
    valores_y_RK_4 = [y_0] # Lista de valores de y
    valores_t = [t_0] # Vector de tiempo

    # Iterar sobre el número de pasos y aplicar el método de Runge-Kutta 4
    for i in range(n_pasos):
        valores_t.append(valores_t[i] + h) # ¡¡¡NO MODIFICAR!!!
        k1 = h * f(valores_y_RK_4[i]) # ¡¡¡NO MODIFICAR!!!
        k2 = h * f(valores_y_RK_4[i] + k1/2) # ¡¡¡NO MODIFICAR!!!
        k3 = h * f(valores_y_RK_4[i] + k2/2) # ¡¡¡NO MODIFICAR!!!
        k4 = h * f(valores_y_RK_4[i] + k3) # ¡¡¡NO MODIFICAR!!!
        y_nueva = valores_y_RK_4[i] + 1/6 * (k1 + 2*k2 + 2*k3 + k4) # ¡¡¡NO MODIFICAR!!!
        valores_y_RK_4.append(y_nueva) # ¡¡¡NO MODIFICAR!!!

    # Graficar
    titulo = f'Caida libre<br>Método de Runge-Kutta 4 con h = {round(h,3)}, n = {n_pasos} y N(0) = {round(y_0,3)}' # ¡¡¡NO MODIFICAR!!!
    eje_x = 't[s]' # Nombre del eje x
    eje_y = 'v(t)[m/s]' # Nombre del eje y

    fig = go.Figure() # ¡¡¡NO MODIFICAR!!!
    fig.update_layout(title=titulo, title_x=0.5, xaxis_title=eje_x, yaxis_title=eje_y) # ¡¡¡NO MODIFICAR!!!
    plot_RK_4 = go.Scatter(x=valores_t, y=valores_y_RK_4, mode='markers', marker=dict(size=8, symbol='triangle-down'), name='Runge-Kutta 4', marker_color='hotpink', showlegend=True) # ¡¡¡NO MODIFICAR!!!
    fig.add_trace(plot_RK_4) # ¡¡¡NO MODIFICAR!!!
    fig.show() # ¡¡¡NO MODIFICAR!!!
    return plot_RK_4 # ¡¡¡NO MODIFICAR!!!

plot_RK_4 = metodo_RK_4(f_real, t_0, y_0, h, n_pasos) # Graficar método de Runge-Kutta 4

## Comparacion de los metodos

In [38]:
titulo = f'Caida libre<br>Comparación de metodos numericos y solucion analitic' # ¡¡¡NO MODIFICAR!!!
eje_x = 't[s]' # Nombre del eje x
eje_y = 'v(t)[m/s]' # Nombre del eje y

fig = go.Figure() # ¡¡¡NO MODIFICAR!!!
fig.update_layout(title=titulo, xaxis_title=eje_x, yaxis_title=eje_y, legend_title='Métodos', title_x=0.5, title_y=0.9) # ¡¡¡NO MODIFICAR!!!
fig.add_trace(plot_Analitica) # ¡¡¡NO MODIFICAR!!!
fig.add_trace(plot_Euler) # ¡¡¡NO MODIFICAR!!!
fig.add_trace(plot_Euler_Mod) # ¡¡¡NO MODIFICAR!!!
fig.add_trace(plot_RK_4) # ¡¡¡NO MODIFICAR!!!

fig.show() # ¡¡¡NO MODIFICAR!!!